In [13]:
include("geqrf.jl")

geqrfMGS! (generic function with 1 method)

In [14]:
function orthogonal_matrix(m,n)
    # Building an orthogonal matrix Q
    Q = zeros(m,n)
    for j=0:n-1, i=0:m-1
        Q[i+1,j+1] = cos(π*(2i+1)*j/2m)
    end
    for j=1:n
        Q[:,j] /= norm(Q[:,j])
    end
    return Q
end    

orthogonal_matrix (generic function with 1 method)

In [15]:
# Testing QR factorization using CGS
n = 4
m = 8

# Building an orthogonal matrix Q
Q = orthogonal_matrix(m,n)

# Initializing an upper triangular matrix R
R = triu(Float64[ i/j for i=1:n, j=1:n ])

# Matrix A
A = Q*R
# QR factorization
RGS = geqrfCGS!(A)
# The factor Q is stored in A

# These matrices should now be equal
@show norm(Q-A)
@show norm(R-RGS)

norm(Q - A) = 4.007085382004466e-16
norm(R - RGS) = 3.2611605820465635e-16


3.2611605820465635e-16

In [16]:
function pretty_print(A)
    m = size(A,1)
    n = size(A,2)
    for i=1:m
        for j=1:n
            @printf "%12.3e" A[i,j]
        end
        @printf "\n"
    end
end    

pretty_print (generic function with 1 method)

In [17]:
@show e = sqrt(eps(Float64))
A0 = [1 1 1; e 0 0; 0 e 0; 0 0 e]
@printf "A\n"
pretty_print(A0)
A = copy(A0); geqrfCGS!(A)
@printf "\nCGS\n"
pretty_print(A)

A = copy(A0); geqrfMGS!(A)
@printf "\nMGS\n"
pretty_print(A)

e = sqrt(eps(Float64)) = 1.4901161193847656e-8
A
   1.000e+00   1.000e+00   1.000e+00
   1.490e-08   0.000e+00   0.000e+00
   0.000e+00   1.490e-08   0.000e+00
   0.000e+00   0.000e+00   1.490e-08

CGS
   1.000e+00   0.000e+00   0.000e+00
   1.490e-08  -7.071e-01  -7.071e-01
   0.000e+00   7.071e-01   0.000e+00
   0.000e+00   0.000e+00   7.071e-01

MGS
   1.000e+00   0.000e+00   0.000e+00
   1.490e-08  -7.071e-01  -4.082e-01
   0.000e+00   7.071e-01  -4.082e-01
   0.000e+00   0.000e+00   8.165e-01


In [18]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [19]:
n = 64
m = 128

Q = orthogonal_matrix(m,n)
R = triu(Float64[ 2.0^(-i)/j for i=1:n, j=1:n ])
# R has decaying entries on the diagonal

A = Q*R
RGS = geqrfCGS!(A);

e = zeros(n)
for j=1:n
    e[j] = norm(Q[:,j]-A[:,j])
end

plot(diag(R),yscale=:log,marker=:auto,left_margin=10mm,lab="exact")
plot!(diag(RGS),marker=:auto,lab="RGS")
plot!(e,marker=:auto,lab="Error in qk")

In [20]:
generate_tex_pgfplot = false 
if generate_tex_pgfplot
    pgfplots()
    plt = plot(diag(R),yscale=:log,marker=:auto,left_margin=10mm)
    plot!(diag(RGS),marker=:auto)
    plot!(e,marker=:auto)
end   

In [21]:
if generate_tex_pgfplot
    PGFPlots.save("CGS_err.tex", plt.o, include_preamble=false)
    plotlyjs()
end

In [22]:
# We use the modified Gram-Schmidt algorithm now
A = Q*R
RGS = geqrfMGS!(A);

e = zeros(n)
for j=1:n
    e[j] = norm(Q[:,j]-A[:,j])
end

plot(diag(R),yscale=:log,marker=:auto,left_margin=10mm,lab="exact")
plot!(diag(RGS),marker=:auto,lab="RGS")
plot!(e,marker=:auto,lab="Error in qk")
# It's way more accurate; we reach machine precision

In [23]:
if generate_tex_pgfplot
    pgfplots()
    plt = plot(diag(R),yscale=:log,marker=:auto,left_margin=10mm)
    plot!(diag(RGS),marker=:auto)
    plot!(e,marker=:auto)
end   

In [24]:
if generate_tex_pgfplot
    PGFPlots.save("MGS_err.tex", plt.o, include_preamble=false)
end